# Bayesian Calibration of Mice Data to ODE System with Treatment

In [5]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner
import scipy.stats
import math
from numba import jit
from tqdm import tqdm
from scipy.integrate import odeint
from scipy.optimize import minimize
from IPython.display import display, Math
from scipy.optimize import Bounds
from multiprocess import Pool
import time
K = 0; # Set Prior K
rPriorMin = 0.0
rPriorMax = 0.0
rMedian = 0.0
aPriorMax = 0.0
STD_MAX = 10

## Defining the model
Let N be the number of tumor cells and assume that this number can increase with a growth rate of $r$ cells/hour. The second term acts as the impact caused by the specific cancer therapy scheme. With these assumptions, the mathematical model is:

$DA=\alpha_R \sum e^{-\beta_R(t-\tau_i)}H(t-\tau_i),
$
- $\boldsymbol{\theta}$: vector of model parameters, $\boldsymbol{\theta}=(r, \alpha_R, \beta_R)$;
- $DA$: Drug Affect;
- $K$: Constant representing Carrying Capacity
- $r$: tumor growth rate;
- $\boldsymbol{\tau_i}$: Initial Dossage Time;
- $\boldsymbol{Y}(\boldsymbol{\theta})$: model prediction;

## Defining the model
Let N be the number of tumor cells and assume that this number can increase with a growth rate of $r$ cells/hour. The second term acts as the impact caused by the specific cancer therapy scheme. With these assumptions, the mathematical model is:

\begin{equation}
\frac{dN}{dt}=rN\left(1-\frac{N}{K}\right)-\sum N\alpha_Re^{-\beta_R(t-\tau_i)}H(t-\tau_i),
\end{equation}

- $\boldsymbol{\theta}$: vector of model parameters, $\boldsymbol{\theta}=(r, \alpha_R, \beta_R)$;
- $K$: Constant representing Carrying Capacity
- $r$: tumor growth rate;
- $\boldsymbol{\tau_i}$: Initial Dossage Time;
- $\boldsymbol{Y}(\boldsymbol{\theta})$: model prediction;

In [6]:
def calibrate(treatment):
    #Getting Mice Data
    data = np.genfromtxt(('../../MiceData/treatment'+str(treatment)+'.csv'), dtype=float, delimiter=',') # Set location of data
    times = data[0,1:-1]
    yobs = data[1:,1:-1]
    n = len(yobs)

    #Creating Labels
    labels = ["β","σ"]
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    for i in range(n):
        labels.insert(0,("r"+str(n-i).translate(SUB)))
        labels.insert(-2-i,("IC"+str(i+1).translate(SUB)))
        labels.insert(-2,("α"+str(i+1).translate(SUB)))

    #Model
    def control_tumor(y, t, theta):
        rval = theta[0] * y[0]*(1-y[0]/K)
        for dtime in ([1,8]):
        # for dtime in ([0,7]):
            if(t-dtime<0):
                return rval
            rval -= y[0]*theta[1]*math.exp(-theta[2]*(t-dtime))
        return rval

    #Likelihood Function
    def log_likelihood_ic(theta, times, y):
        ll = 0
        for r in range(n):
            model = odeint(control_tumor, y0=theta[n+r], t=times, args=tuple([[theta[r],theta[r+2*n],theta[3*n]]]))
            variance = theta[3*n+1]**2
            model.shape = y[r,:].shape
            ll += -0.5 * np.sum((y[r,:] - model) ** 2 / variance + np.log(2*np.pi) + np.log(variance))
        return ll

    #Setting up Priors
    def log_prior(theta):
        for r in range(3*n+2):
            if r<n:
                #r Vals
                if rPriorMin<theta[r]<rPriorMax:
                    continue
                else:
                    return -np.inf
            elif r<(2*n):
                #Initial Conditions
                if 0<theta[r]<600:
                    continue
                else:
                    return -np.inf
            elif r<(3*n):
                #Alpha Conditions
                if 0<theta[r]<aPriorMax:
                    continue
                else:
                    return -np.inf
            elif r==(3*n):
                #Beta Conditions
                if 0<theta[r]<2:
                    continue
                else:
                    return -np.inf
            elif r==(3*n+1):
                #Standard deviation
                if 0.1<theta[r]<STD_MAX:
                    continue
                else:
                    return -np.inf
        return 0.0

    def log_probability(theta, times, y):
        lp = log_prior(theta)
        if not np.isfinite(lp):
            return -np.inf
        return lp + log_likelihood_ic(theta, times, y)

    #Creating mean_par array:
    mean_par = np.concatenate([np.full(n,rMedian), yobs[:,0], np.full(n,0.5), np.array([0.5,STD_MAX/2]),])
    #Treatment 2 Priors:
    pos = mean_par * (1 + 5e-2 * np.random.randn((2*(3*n+2)),(3*n+2)))
    #^ change number of chains
    nwalkers, ndim = pos.shape

    with Pool() as pool:
        start = time.time();
        sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, pool=pool, args=(times, yobs))
        sampler.run_mcmc(pos, 20000, progress=True);
        end = time.time()
        print("Multiprocessing took {0:.1f} seconds".format(end-start))

    flat_samples = sampler.get_chain(discard=3000, flat=True)

    def mean_ci(data, confidence=0.95):
        n = len(data)
        m = np.mean(data)
        se = scipy.stats.sem(data)
        h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
        return m, h

    fig, axes = plt.subplots(nrows=(3*n+2),ncols=1, figsize=(15, 20), sharex=True)
    axes.shape = (3*n+2)
    samples = sampler.get_chain()
    for i in range(ndim):
        ax = axes[i]
        ax.plot(samples[:, :, i], "k", alpha=0.3)
        ax.set_xlim(0, len(samples))
        ax.set_ylabel(labels[i])
        ax.yaxis.set_label_coords(-0.1, 0.5)
    axes[-1].set_xlabel("step number")
    axes[-2].set_xlabel("step number")
    plt.savefig("chainConvergence" + str(treatment) + ".pdf")

    for r in range(n):
        fig, ax = plt.subplots(ncols=2,figsize=(8,3),dpi=150)
        inds = np.random.randint(len(flat_samples), size=50)
        create = True
        for ind in inds:
            sample = flat_samples[ind]
            y_sp = odeint(control_tumor, y0=sample[n+r], t=times, args=tuple([[sample[r],sample[r+2*n],sample[3*n]]]))
            if create:
                A = y_sp
                create = False
            else:
                A = np.append(A,y_sp,1)
            ax[0].plot(times, y_sp, 'blue', alpha=0.5)
        ax[0].plot(times, y_sp, label='Calibrated model', color='blue', alpha=0.5)
        ax[0].scatter(times, yobs[r], zorder=100, label='Measured data', color='red')
        ax[0].legend(frameon=False)
        ax[0].set_xlabel('Time (hours)')
        ax[0].set_ylabel('Cell number')

        mean = []
        ci = []
        for b in range(A.shape[0]):
            m = mean_ci(A[b,:],0.95)
            mean.append(m[0])
            ci.append(m[1])
        ci = np.array(ci)
        mean = np.array(mean)

        # Plot the figure
        ax[1].fill_between(times, mean - ci, mean + ci, alpha = 0.25, color = 'blue',label='95% CI')
        ax[1].plot(times, mean - ci, linewidth = 1, linestyle = "--", color = 'blue')
        ax[1].plot(times, mean + ci, linewidth = 1, linestyle = "--", color = 'blue')
        #ax.plot(times, mean,"-o", color = 'blue')
        ax[1].scatter(times, yobs[r], zorder=100, label='Measured data', color='red')
        ax[1].legend(frameon=False)
        ax[1].set_xlabel('Tumor volume (mm³)')
        ax[1].set_ylabel('Days Post Tumor Implantation')
        #plt.savefig("bc_fit.pdf")
        plt.show()
    return flat_samples

In [ ]:
# Compute calibration
for c in range(5):
    treatment = calibrate(c+1)
    np.savetxt("FullCalibration/flatSamplesTreatment" + str(c+1) + ".csv", treatment, delimiter=",");

In [ ]:
# Produce Corner diagram to test for convergence
labels = ["β","σ"]
n=4
SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
for i in range(n):
    labels.insert(0,("r"+str(n-i).translate(SUB)))
    labels.insert(-2-i,("IC"+str(i+1).translate(SUB)))
    labels.insert(-2,("α"+str(i+1).translate(SUB)))
fig = corner.corner(treatment, labels=labels)